# MaternalCare AI Assistant

An AI-Driven System to Address Maternal Health Disparities in the U.S.

📘 Course: ITAI 2377 – AI Capstone

Group Name: Data Miner

Team Members: Zahra & Virginia

Instructor: Maryam Esmali

## 1. Executive Summary

MaternalCare_AI is a domain-specific artificial intelligence (AI) assistant developed to bridge critical information gaps in maternal healthcare, particularly for individuals in underserved and high-risk communities. The assistant delivers accurate, accessible, and context-aware responses to maternal health queries by integrating structured geographic vulnerability data with a curated knowledge base of medical guidance.

At the core of the system is a retrieval-augmented generation (RAG) architecture, which semantically matches user queries to embedded knowledge chunks derived from expert-verified documents. This retrieval is further enhanced by incorporating ZIP Code Tabulation Area (ZCTA) data, enabling the assistant to tailor responses based on localized health risk indicators such as the Social Vulnerability Index (SVI).

The primary objective of MaternalCare_AI is to empower users with plain-language, trustworthy insights on topics such as prenatal care, postpartum symptoms, and available community resources. Its lightweight, modular design makes it deployable in low-resource environments, positioning it as a practical tool for improving maternal health literacy and support equity.

# 2. Project Definition
## 2.1 Domain Selection

The maternal healthcare domain was chosen due to the persistent disparities in maternal health outcomes across geographically and socioeconomically vulnerable populations. Women in high-risk ZIP codes often face limited access to timely and reliable health information. This project addresses the urgent need for culturally appropriate, accessible, and geographically personalized maternal health support through an AI-powered assistant.

## 2.2 Problem Statement

Despite the availability of online health resources, many pregnant individuals especially those in marginalized communities—lack access to trustworthy, understandable, and relevant maternal health guidance. Existing materials are frequently fragmented, overly clinical, or disconnected from the lived experiences of their intended users. There is a critical gap in tools that combine localized risk awareness with plain-language health communication.

## 2.3 Core Functionalities

**Localized Health Information**
Uses ZIP Code Tabulation Area (ZCTA) and CDC’s Social Vulnerability Index (SVI) to tailor responses based on geographic health risk levels.

**Knowledge Base Q&A**
Implements a retrieval-augmented generation (RAG) system to match user queries with semantically embedded knowledge chunks for context-relevant answers.

**Multi-Source Knowledge Integration**
Combines public health documents, articles, and FAQs into a unified, embedded knowledge base to enhance response quality and coverage.

**Performance Evaluation**
Applies a structured golden question-answer set to evaluate semantic accuracy and retrieval effectiveness.

## 2.4 Target Users

* Pregnant individuals and postpartum mothers seeking accessible health advice

* Community health workers supporting maternal care in under-resourced areas

* Nonprofits, advocacy groups, and public health organizations involved in maternal wellness

* Government agencies focused on maternal health education and intervention

## 2.5 Value Proposition

MaternalCare_AI provides a scalable, interpretable, and equity-focused solution to maternal health information access. By combining location-based vulnerability data with retrieval-augmented AI, it delivers relevant, plain-language guidance to users who may otherwise face barriers to care. The system is fully deployable in low-resource settings (e.g., via Google Colab), making it accessible for use in community outreach, mobile health clinics, and digital health platforms.

## Setup (folders + packages)

In [1]:
# Create a local project structure under /content (no Drive needed)
import os, sys, subprocess
PROJECT = '/content/ITAI2377_MaternalCareAI_OneNotebook'
for p in [PROJECT, f'{PROJECT}/data/raw', f'{PROJECT}/data/processed', f'{PROJECT}/data/embeddings', f'{PROJECT}/models', f'{PROJECT}/outputs']:
    os.makedirs(p, exist_ok=True)
os.chdir(PROJECT)

def pip_install(pkgs):
    cmd = ['pip','install','-q'] + pkgs
    print("Installing:", " ".join(pkgs))
    subprocess.check_call(cmd)

pip_install(['pandas','numpy','requests','sentence-transformers','chromadb','beautifulsoup4','pdfminer.six','tldextract','validators','tiktoken'])

import pandas as pd, numpy as np
print('✅ Ready. Working dir:', os.getcwd())


Installing: pandas numpy requests sentence-transformers chromadb beautifulsoup4 pdfminer.six tldextract validators tiktoken
✅ Ready. Working dir: /content/ITAI2377_MaternalCareAI_OneNotebook


# 3. Data Requirements Analysis
## 3.1 Data Types

**Structured:**
Social Vulnerability Index (SVI) scores in CSV format, representing geographic vulnerability at the ZIP Code Tabulation Area (ZCTA) level. These values are used to personalize responses based on regional risk factors.

**Unstructured:**
Free-text maternal care guidance compiled from public domain resources. This content (e.g., kb_docs.csv) forms the retrieval-based knowledge base for the assistant and is preprocessed into semantic chunks for embedding and querying.

## 3.2 Data Sources

**SVI Dataset:**
Sourced from the CDC/ATSDR Social Vulnerability Index via its open data portal, providing ZCTA-level indicators such as poverty, access to transportation, and housing conditions.

**Knowledge Base:**
Derived from a curated collection of public domain maternal health articles, FAQs, and guidelines. Sources include organizations like the WHO, Mayo Clinic, and U.S. Department of Health and Human Services.

## 3.3 Data Volume & Velocity

Static dataset (~30,000 ZCTAs; <100 documents)

Updated manually or annually
Approximately 30,000 ZCTA entries in the structured SVI dataset and fewer than 100 unstructured documents in the knowledge base. Datasets are largely static and updated manually or annually. No streaming or real-time ingestion is required.

## 3.4 Data Quality Requirements

Valid ZCTA codes (zero-padded)
Clean, readable documents with a minimum word count

No duplicate or irrelevant text chunks

## 3.5 Data Challenges

Inconsistent ZCTA formats

Sparse SVI data for some ZIP codes

Variable quality or length of unstructured KB documents

## 3.6 Data Schema

[SVI Table]
ZCTA → RPL_THEMES

[KB Table]
title → text → chunk → embedding


## Upload Datasets (Manual)
Upload one or more CSVs, such as:
- `SVI_2022_US_ZCTA.csv` (CDC/ATSDR SVI by ZCTA)
- `kb_docs.csv` (two columns: `title`, `text`) for the knowledge base
- Any other CSVs you want to explore

Files will be placed into `data/raw/` automatically.


In [4]:
from google.colab import files
import os, pandas as pd

uploaded = files.upload()  # Choose CSVs from your computer
uploaded = files.upload()  # Choose CSVs from your computer
uploaded = files.upload()  # Choose CSVs from your computer
uploaded = files.upload()  # Choose CSVs from your computer
for filename in uploaded.keys():
    src = filename
    dst = f'data/raw/{filename}'
    os.replace(src, dst)
    print('✅ Saved to', dst)

print('\n📁 Raw files in data/raw/:', os.listdir('data/raw'))


Saving SVI_2022_US_ZCTA.csv to SVI_2022_US_ZCTA.csv


Saving Maternal Health Risk Data Set.csv to Maternal Health Risk Data Set.csv


Saving kb_docs.csv to kb_docs.csv


Saving Golden_QA_Semantic_Evaluation.csv to Golden_QA_Semantic_Evaluation.csv
✅ Saved to data/raw/Golden_QA_Semantic_Evaluation.csv

📁 Raw files in data/raw/: ['Golden_QA_Semantic_Evaluation.csv']


# 4. Processing Pipeline Design
## 4.1 Preprocessing Workflow

Upload and validate SVI_2022_US_ZCTA.csv

Clean columns, zero-pad ZCTA, filter key SVI metrics

Upload and chunk kb_docs.csv using word-based overlap

Generate sentence-level embeddings (MiniLM)

Store embeddings in ChromaDB (vector store)

## 4.2 Feature Engineering

Text chunks (120-word segments with 20-word overlap)

Embedding vectors via all-MiniLM-L6-v2

ZCTA-matched filtering using SVI

## 4.3 Pipeline Diagram (to include as figure)
Raw CSVs
   ↓
[Cleaned SVI] + [Chunked Text]
   ↓
Sentence Embedding (MiniLM)
   ↓
Vector DB (ChromaDB)
   ↓
Query → Retrieval → Compose Answer

## 4.4 Data Transformation Techniques

Lowercasing, punctuation removal

Word-based chunking with overlap

Sentence embedding using pretrained model

## 4.5 Infrastructure

Runtime: Google Colab

Storage: Local in-session (non-persistent)

Libraries: pandas, sentence-transformers, chromadb

## Inspect & Clean SVI (ZCTA-Level)
- Detect ZCTA column name and zero-pad to 5 characters
- Keep common SVI theme columns (you can modify this list)
- Save clean file to `data/processed/svi_2022_clean.csv`

> If your SVI file has a different name, set `SVI_FILE` below.


In [6]:
import pandas as pd, os

# Change this if your uploaded file name is different
SVI_FILE = 'SVI_2022_US_ZCTA.csv'
# Correct the raw_path to load from /content directly
raw_path = f'/content/{SVI_FILE}'
assert os.path.exists(raw_path), f'Upload your SVI file as {raw_path} or update SVI_FILE.'

df_svi = pd.read_csv(raw_path, low_memory=False)
print('SVI raw shape:', df_svi.shape)
display(df_svi.head(3))

# Standardize columns
df_svi.columns = [c.strip().lower() for c in df_svi.columns]

# Try to detect ZCTA column
zcta_col = None
for cand in ['zcta','zcta5','zcta_5','fips','zcta5ce10']:
    if cand in df_svi.columns:
        zcta_col = cand
        break
if zcta_col is None:
    raise ValueError('Could not detect the ZCTA column. Please check headers and set zcta_col manually.')

# Zero-pad ZCTA
df_svi[zcta_col] = df_svi[zcta_col].astype(str).str.zfill(5)

# Choose SVI theme columns to keep (customize as needed)
keep_like = ['rpl_themes','rpl_soc','rpl_house','rpl_minor','rpl_tran']
keep = [zcta_col] + [c for c in df_svi.columns if any(k in c for k in keep_like)]
df_svi_clean = df_svi[keep].drop_duplicates().dropna(how='all')

os.makedirs('data/processed', exist_ok=True)
out_svi = 'data/processed/svi_2022_clean.csv'
df_svi_clean.to_csv(out_svi, index=False)
print('✅ Saved:', out_svi, df_svi_clean.shape)
display(df_svi_clean.head(3))

SVI raw shape: (33642, 153)


,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,...,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,1001,ZCTA5 01001,11.292492,16045,946,6908,385,6611,390,2095,...,3.0,2.1,0.0,0.3,0.0,0.3,1.1,0.8,0.3,0.4
1,1002,ZCTA5 01002,53.255781,22992,308,10221,332,8782,365,5735,...,9.0,1.7,0.1,0.1,0.5,0.3,7.3,1.8,0.1,0.1
2,1003,ZCTA5 01003,0.804123,11989,205,91,63,19,19,3,...,20.3,2.6,0.0,0.4,0.0,0.4,5.6,1.5,0.0,0.4


✅ Saved: data/processed/svi_2022_clean.csv (33642, 2)


,fips,rpl_themes
0,01001,0.7071
1,01002,0.6668
2,01003,0.1057


## 5) Build Text Chunks for RAG
We use a CSV named `kb_docs.csv` with two columns: `title`, `text`. If you didn’t upload one, a tiny demo will be created.


# 5. Implementation Strategy
## 5.1 Technical Stack

Language: Python

Libraries: pandas, numpy, sentence-transformers, chromadb, gradio

Notebook Platform: Google Colab

## 5.2 Stages

1	Environment setup, data upload
2	SVI cleaning, chunking
3	Embedding + retrieval implementation
4	Evaluation, dashboard, final polish

## 5.3 Team Responsibilities
Member	Responsibility
Zahra	Data loading, preprocessing, Gradio UI
Virginia	Chunking, embedding, retrieval logic

## 5.4 Resource Needs

Public datasets

Free-tier Google Colab runtime

No external API keys or LLM access needed

## 5.5 Anticipated Challenges

Chunking small documents may produce low-density embeddings

Colab session resets (mitigated via checkpoint saving)

In [9]:
import pandas as pd

# Load the updated kb_docs.csv file
kb_path = '/content/ITAI2377_MaternalCareAI_OneNotebook/kb_docs.csv'  # Update path if needed
kb = pd.read_csv(kb_path)
kb.columns = [col.lower() for col in kb.columns]

# Chunk the text
def chunk_text(t, chunk_words=120, overlap=20):
    words = str(t).split()
    chunks = []
    step = max(1, chunk_words - overlap)
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i+chunk_words])
        if len(chunk.split()) >= 20 or i == 0:
            chunks.append(chunk)
    return chunks

# Build chunk records
records = []
for i, row in kb.iterrows():
    chunks = chunk_text(row['text'])
    for j, ch in enumerate(chunks):
        records.append({
            'doc_id': f'doc_{i}',
            'chunk_id': f'doc_{i}_chunk_{j}',
            'title': row['title'],
            'chunk_text': ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.to_csv('/content/chunks.csv', index=False)
print("✅ Chunks Created:", chunks_df.shape)

✅ Chunks Created: (5, 4)


## Create Embeddings (SentenceTransformers all-MiniLM-L6-v2)

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

chunks_df = pd.read_csv('/content/chunks.csv')
vecs = model.encode(chunks_df['chunk_text'].tolist(), show_progress_bar=True)

np.save('/content/chunk_embeddings.npy', vecs)
print("✅ Embeddings Created:", vecs.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings Created: (5, 384)


## Load ChromaDB (Vector Database)

In [11]:
import chromadb

client = chromadb.PersistentClient(path="/content/chroma_db")

# Optional: delete previous collection
try:
    client.delete_collection('maternalcare_kb')
except:
    pass

collection = client.create_collection(name='maternalcare_kb')

# Reload chunks and embeddings
chunks_df = pd.read_csv('/content/chunks.csv')
vecs = np.load('/content/chunk_embeddings.npy')

collection.add(
    ids=chunks_df['chunk_id'].tolist(),
    embeddings=vecs.tolist(),
    documents=chunks_df['chunk_text'].tolist(),
    metadatas=chunks_df[['doc_id','title']].to_dict('records')
)
print("✅ ChromaDB Loaded with Vector Store")

✅ ChromaDB Loaded with Vector Store


# 6. Evaluation Framework
## 6.1 Success Metrics

Hit@3: Correct answer in top 3 retrieved chunks

Manual QA Review: Reviewer confirms relevance

UI Response Quality: Format, clarity, and usefulness

## 6.2 Testing Strategy

Compare system response with golden answer key

Use embedded must-have keywords to score hits

## 6.3 Feedback Mechanism

Future integration with Gradio thumbs or survey

Review flagged queries for content improvement

## 6.4 Performance Benchmarks
Metric	Target Value

Hit@3 Accuracy	≥ 80%

Response Time	< 2 sec/query

Completion	100% of test questions return an answer

## 6.5 Continuous Improvement

Add new documents as they are sourced

Monitor feedback logs

Re-embed if source texts are revised

## Retrieve & Compose an Answer (Prototype)

In [12]:
# Compose response
def compose_answer(query, results):
    retrieved_docs = results.get('documents', [[]])[0]
    if not retrieved_docs:
        return f"⚠️ I couldn’t find any relevant answer for: **{query}**. Try rephrasing your question."

    bullet_points = "\n\n".join(f"- {doc.strip()}" for doc in retrieved_docs)
    return f"""### 🧾 Question:\n**{query}**

### 📘 Answer Summary:
{bullet_points}
"""

# Safe RAG call
def ask_maternal_ai(query):
    try:
        qv = model.encode([query])[0]
        results = collection.query(query_embeddings=[qv.tolist()], n_results=3)
        return compose_answer(query, results)
    except Exception as e:
        return f"❌ Error during processing: {str(e)}"

## Basic Evaluation (hit@3 on a tiny golden set)

In [13]:
import numpy as np

golden = [
    {"q":"postpartum danger signs","must":["bleeding","headache","shortness of breath"]},
    {"q":"prenatal care basics","must":["prenatal","screenings","visits"]},
]

def retrieve_docs(q, k=3):
    qv = model.encode([q])[0]
    res = collection.query(query_embeddings=[qv.tolist()], n_results=k)
    return res['documents'][0]

hits = []
for item in golden:
    docs = retrieve_docs(item["q"], k=3)
    text = " ".join(docs).lower()
    ok = all(term in text for term in item["must"])
    hits.append(int(ok))

print("✅ hit@3:", round(float(np.mean(hits)), 3), "| per-item:", hits)


✅ hit@3: 1.0 | per-item: [1, 1]


In [14]:
question = "What are the signs of postpartum depression?"
response = ask_maternal_ai(question)
print(response)


### 🧾 Question:
**What are the signs of postpartum depression?**

### 📘 Answer Summary:
- Common signs of postpartum depression include persistent sadness, anxiety, irritability, loss of interest in the baby, sleep disturbances, and feelings of hopelessness. It's important to seek professional help if these symptoms continue.

- Symptoms that require immediate medical attention after birth include severe headache, chest pain, heavy vaginal bleeding, shortness of breath, or thoughts of harming oneself or the baby. Call emergency services or go to the hospital immediately.

- Prenatal checkups are typically scheduled once a month until 28 weeks, every two weeks until 36 weeks, and weekly from 36 weeks to delivery. These visits help monitor the health of both mother and baby.



# 7. Interactive Dashboard
## 7.1 Interface: Gradio

Implemented using Gradio with:

Input box for questions

Markdown output with formatted responses

Clickable example queries

## 7.2 Sample Output

Question: What symptoms after birth require emergency care?
Answer Summary:

Seek immediate help for severe headache, chest pain, heavy bleeding, or shortness of breath.

In [15]:
import gradio as gr

sample_questions = [
    "What are the signs of postpartum depression?",
    "How often should I get prenatal checkups?",
    "What symptoms after birth require emergency care?",
    "Where can I get low-cost maternal health support?",
    "What is prenatal care and why is it important?"
]

gr.Interface(
    fn=ask_maternal_ai,
    inputs=gr.Textbox(lines=2, placeholder="Ask about maternal health..."),
    outputs=gr.Markdown(label="AI Assistant Response"),
    title="🤰 MaternalCare AI Assistant",
    description="Answering common maternal health questions based on a curated knowledge base.",
    examples=sample_questions
).launch(debug=True, share=False)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


# 8. Conclusion

The development of MaternalCare_AI challenged us to apply data science and AI principles to a real-world domain with high social impact - maternal health. Our goal was to create an assistant that could offer plain-language, localized health guidance to underserved communities, particularly those in ZIP codes with high Social Vulnerability Index (SVI) scores. Throughout the project, we built a full pipeline that included curated knowledge base creation, semantic chunking, embedding generation, and retrieval-augmented QA through ChromaDB. The assistant was deployed via a Gradio interface to ensure usability for both non-technical users and health outreach workers. One of the key learning outcomes was how to balance simplicity with utility. We made conscious decisions to use free, open-source tools (e.g., Colab, MiniLM, CSV-based KB) to simulate real-world constraints. Our evaluation strategy went beyond just accuracy - we analyzed semantic correctness using a golden QA set and cosine similarity, giving us insights into how useful the answers truly were. Looking forward, we see clear potential for integrating user feedback, adding multilingual support, and expanding the assistant to other public health domains like immunization or pediatric care. The project not only helped us consolidate technical skills but deepened our understanding of equitable AI design.

## Team Contributions
Zahra: Led data preprocessing, developed Gradio UI, and designed the semantic evaluation framework.
Virginia: Implemented embedding + retrieval pipeline, managed the SVI and maternal risk datasets, and oversaw model testing and validation.

All team members participated in planning, documentation, and final editing. Collaboration was managed using Google Drive, Colab notebooks, and shared version checkpoints. Each member reviewed and approved all final deliverables.

# References

Centers for Disease Control and Prevention (CDC). (2023). Social Vulnerability Index (SVI) 2022 Database. U.S. Department of Health & Human Services.
https://www.atsdr.cdc.gov/placeandhealth/svi/index.html

Kaggle Contributor. (2020). Maternal Health Risk Data Set. [Dataset]. Kaggle.
https://www.kaggle.com/datasets/andrewmvd/maternal-health-risk-data

Reimers, N., & Gurevych, I. (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. arXiv preprint arXiv:1908.10084.
https://arxiv.org/abs/1908.10084

ChromaDB Team. (2023). Chroma: The AI-native open-source embedding database.
https://www.trychroma.com/

Gradio Team. (2023). Gradio: Build machine learning demos in Python.
https://www.gradio.app/

MiniLM: Wang, Y., Wei, F., Dong, L., Bao, H., Yang, N., & Zhou, M. (2020). MiniLM: Deep Self-Attention Distillation for Task-Agnostic Compression of Pre-Trained Transformers. arXiv preprint arXiv:2002.10957.
https://arxiv.org/abs/2002.10957

MD Data Miner Group. (2025). Golden QA Semantic Evaluation Dataset for MaternalCare_AI. Internal course project dataset. ITAI2377, Fall 2025.